Based on: https://github.com/udacity/self-driving-car-sim

In [5]:
from flask import Flask
import eventlet.wsgi
import eventlet
import socketio
import base64
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt # to show the image
from keras.models import load_model 

In [6]:
sio = socketio.Server()
app = Flask(__name__)

# function to control the car
def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
'throttle': str(throttle) }, skip_sid=True)

### Load the downloaded model

In [7]:
model = load_model('mymodel.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalize (Lambda)           (None, 60, 80, 3)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 58, 78, 64)        1792      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 56, 76, 32)        18464     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 26, 36, 16)        4624      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 18, 16)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 11, 16, 8)         1160      
__________

In [8]:
# function to process image
def process_image(img):
    # we are deleting unused pixels (the top and the bottom of the image and we are taking every second pixel)
    return img[10:130:2,::4,:]

# function to receive data from simulator
@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]
        
        decode = base64.b64decode(image_str) # decode the image_str (instead of string we have bytes)
        image = Image.open(BytesIO(decode)) # from bytes to image
        image_array = np.asarray(image) # from image to numpy array
        
        # to show the image, which is 'seeing' by car
        #plt.imshow(image_array)
        #plt.show();
        
        img = process_image(image_array)
        # we expend numer of dimensions (+1)
        img_batch = np.expand_dims(img, axis=0)
        
        # steering_angle now is coming our (downloaded) model
        steering_angle = float(model.predict(img_batch))
        
        #steering_angle = -1.0        # angle <-1.0 .. 1.0>
        throttle = 0.15              # < -1.0 .. 1.0> 
        
        if speed < 13:
            throttle = 0.5
        if speed > 15:
            throttle = -0.1
        
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)

app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(6395) wsgi starting up on http://0.0.0.0:4567
wsgi exiting
(6395) wsgi exited, is_accepting=True
